In [2]:
#!pip install transformers

In [23]:
from transformers import pipeline

#zero-shot sequence classifiers

Yin et al. proposed a method for using pre-trained NLI models as a ready-made zero-shot sequence classifiers. The method works by posing the sequence to be classified as the NLI premise and to construct a hypothesis from each candidate label.

In [24]:
classifier = pipeline("zero-shot-classification",model="facebook/bart-large-mnli")

#zero-shot classification method:
It is surprisingly effective in many cases, particularly when used with larger pre-trained models like BART and Roberta. 

Here,we are using this model for zero-shot classification with Hugging Face's built-in pipeline

In [25]:
import pandas as pd

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
df = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/SECONDYEAR_TRIMESTER4_Module03_deep-learning-applications/DAY1_APPLICATION_OF_DEEP_LEARNING/amazon_review.csv')

In [27]:
df.head(5)

,Score,Text
0,5,I have bought several of the Vitality canned d...
1,1,Product arrived labeled as Jumbo Salted Peanut...
2,4,This is a confection that has been around a fe...
3,2,If you are looking for the secret ingredient i...
4,5,Great taffy at a great price. There was a wid...


In [28]:
df=df[(df.Score == 1) | (df.Score == 5)]

In [29]:
df.head(5)

,Score,Text
0,5,I have bought several of the Vitality canned d...
1,1,Product arrived labeled as Jumbo Salted Peanut...
4,5,Great taffy at a great price. There was a wid...
6,5,This saltwater taffy had great flavors and was...
7,5,This taffy is so good. It is very soft and ch...


In [30]:
df.Score = df.Score.replace(5,'Excellent,Awesome,great,outstanding,brilliant')
df.Score = df.Score.replace(1,'bad,poor,unworthy,below average,unsatisfactory')

In [31]:
df.head(5)

,Score,Text
0,"Excellent,Awesome,great,outstanding,brilliant",I have bought several of the Vitality canned d...
1,"bad,poor,unworthy,below average,unsatisfactory",Product arrived labeled as Jumbo Salted Peanut...
4,"Excellent,Awesome,great,outstanding,brilliant",Great taffy at a great price. There was a wid...
6,"Excellent,Awesome,great,outstanding,brilliant",This saltwater taffy had great flavors and was...
7,"Excellent,Awesome,great,outstanding,brilliant",This taffy is so good. It is very soft and ch...


In [32]:
df['Score'].unique()

array(['Excellent,Awesome,great,outstanding,brilliant',
       'bad,poor,unworthy,below average,unsatisfactory'], dtype=object)

In [33]:
df.shape

(409871, 2)

In [34]:
amazon_db = df.sample(frac =.00025)

In [35]:
amazon_db.shape

(102, 2)

In [36]:
sequences=amazon_db['Text']

In [37]:
Score_list = amazon_db['Score'].tolist()
genlist=[]
for k in range(0,100):
  x = Score_list[k].split(",")
  for i in x:
    genlist.append(i)
set_Genre = set(genlist)
genre = (list(set_Genre))

#Applying Zero Shot Text Classification Model on IMDB-Movie-Data Dataset:

we will collect first 100 rows of IMDB-Movie-Data Dataset and
use Zero Shot Text Classification Model on the “Description” Field to predict the “Genre” Labels.

In [38]:
print(genre)

['great', 'bad', 'unsatisfactory', 'Excellent', 'poor', 'below average', 'brilliant', 'Awesome', 'outstanding', 'unworthy']


In [39]:
sequences[0:5]

318086    I love this tea.  It has great flavor.  It rea...
378558    My wife likes the lower acidity of this coffee...
87517     Nespresso makes GREAT coffee and GREAT machine...
15369     OK...I's serious here. If you like Cajun food ...
5505      I just picked this up at my local grocery stor...
Name: Text, dtype: object

#predicting the “Genre” Labels

In [42]:
x=[]
j=0
for i in sequences:
  x.append(classifier(i,genre,multi_label=True))
  if(j<10):
    print(x[j])
  j=j+1
with open("classification_100.txt","w")as file1:
  file1.write(str(x))
file1.close()

{'sequence': "I love this tea.  It has great flavor.  It really beats all the tea bags I've been using.  Add some fruit while brewing for extra flavor.", 'labels': ['outstanding', 'great', 'Excellent', 'brilliant', 'Awesome', 'below average', 'unworthy', 'poor', 'bad', 'unsatisfactory'], 'scores': [0.9987502694129944, 0.9982247352600098, 0.9972559213638306, 0.9945630431175232, 0.9938384890556335, 0.0009115230641327798, 0.0004369163070805371, 0.00041845536907203496, 0.00041059424984268844, 0.00013638348900713027]}
{'sequence': 'My wife likes the lower acidity of this coffee and the flavor is good. This was the best price I have found for this coffee as well which is a bonus. If higher acidity coffees impact you unfavorably I recommend trying this brand.', 'labels': ['great', 'Excellent', 'outstanding', 'Awesome', 'brilliant', 'below average', 'bad', 'unworthy', 'poor', 'unsatisfactory'], 'scores': [0.9765316843986511, 0.9220234155654907, 0.8609794974327087, 0.8419145345687866, 0.6349940

# Evaluating prediction probabilities for the first five sequences:

predicting the “Genre” Labels,we are listing all scores of different labels for first five sequences and comparing with what is available in Genre_list.we get prediction probabilities of quite many labels as higher than 45% and mostly in the range of 70-90% which is quite satisfactory. 

In [43]:
for i in range(0,5):
  print(x[i]['sequence'])
  print("Score_list:",genre)
  loop_len=len(x[i]['labels'])
  for j in range(0,loop_len):
    if(x[i]['scores'][j]<0.20):
      continue
    print(x[i]['labels'][j],'.....',x[i]['scores'][j])

I love this tea.  It has great flavor.  It really beats all the tea bags I've been using.  Add some fruit while brewing for extra flavor.
Score_list: ['great', 'bad', 'unsatisfactory', 'Excellent', 'poor', 'below average', 'brilliant', 'Awesome', 'outstanding', 'unworthy']
outstanding ..... 0.9987502694129944
great ..... 0.9982247352600098
Excellent ..... 0.9972559213638306
brilliant ..... 0.9945630431175232
Awesome ..... 0.9938384890556335
My wife likes the lower acidity of this coffee and the flavor is good. This was the best price I have found for this coffee as well which is a bonus. If higher acidity coffees impact you unfavorably I recommend trying this brand.
Score_list: ['great', 'bad', 'unsatisfactory', 'Excellent', 'poor', 'below average', 'brilliant', 'Awesome', 'outstanding', 'unworthy']
great ..... 0.9765316843986511
Excellent ..... 0.9220234155654907
outstanding ..... 0.8609794974327087
Awesome ..... 0.8419145345687866
brilliant ..... 0.6349940896034241
Nespresso makes GR

In [40]:
!jupyter nbconvert --to html FB_bart_large_mnli_zero_shot.ipynb

[NbConvertApp] Converting notebook FB_bart_large_mnli_zero_shot.ipynb to html
[NbConvertApp] Writing 647626 bytes to FB_bart_large_mnli_zero_shot.html
